<a href="https://colab.research.google.com/github/damoralesr/MetNumUN2022II/blob/main/Lab8/week1LUpivotingGroup18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GRUPO 18**

# I. $LU$ factorization of a square matrix
When we premultply $A$ by lower triangular elemetary matrices $\Lambda_j$ it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  = U$$

$$  A  = (\Lambda_{n-1}  \ldots\Lambda_2  \Lambda_2 )^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses
$$  A  = (\Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1}) U$$

and inverse of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_j^{-1} = - \Lambda_j$, so 

$$ A  =  (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  $$
an upper triangular matrix
$$ L  =  \Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} m = (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  $$
an lower triangular matrix.

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [ ]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [ ]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 8
A = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        A[i, j] = 3. / (0.85*i*j + 1)

np.linalg.matrix_rank(A)

8

In [ ]:
np.round(A,3)

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 1.622, 1.111, 0.845, 0.682, 0.571, 0.492, 0.432],
       [3.   , 1.111, 0.682, 0.492, 0.385, 0.316, 0.268, 0.233],
       [3.   , 0.845, 0.492, 0.347, 0.268, 0.218, 0.184, 0.159],
       [3.   , 0.682, 0.385, 0.268, 0.205, 0.167, 0.14 , 0.121],
       [3.   , 0.571, 0.316, 0.218, 0.167, 0.135, 0.113, 0.098],
       [3.   , 0.492, 0.268, 0.184, 0.14 , 0.113, 0.095, 0.082],
       [3.   , 0.432, 0.233, 0.159, 0.121, 0.098, 0.082, 0.07 ]])

In [ ]:
L, U = diy_lu(A)

print(np.round(L,3), "\n")
print(np.round(U,3), "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(np.round(L@U - A,3))

[[1.    0.    0.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.    0.    0.   ]
 [1.    1.37  1.    0.    0.    0.    0.    0.   ]
 [1.    1.563 1.646 1.    0.    0.    0.    0.   ]
 [1.    1.682 2.077 1.949 1.    0.    0.    0.   ]
 [1.    1.762 2.382 2.731 2.273 1.    0.    0.   ]
 [1.    1.82  2.608 3.364 3.532 2.609 1.    0.   ]
 [1.    1.863 2.783 3.879 4.687 4.475 2.953 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00
   3.000e+00  3.000e+00]
 [ 0.000e+00 -1.378e+00 -1.889e+00 -2.155e+00 -2.318e+00 -2.429e+00
  -2.508e+00 -2.568e+00]
 [ 0.000e+00  0.000e+00  2.700e-01  4.450e-01  5.610e-01  6.440e-01
   7.050e-01  7.520e-01]
 [ 0.000e+00 -0.000e+00  0.000e+00 -1.600e-02 -3.200e-02 -4.500e-02
  -5.500e-02 -6.300e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.000e-03
   1.000e-03  2.000e-03]
 [ 0.000e+00 -0.000e+00  0.000e+00  0.000e+00 -0.000e+00 -0.000e+00
  -0.000e+00 -0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [ ]:
A1 = A.copy()
A1[1, 1] = 3

In [ ]:
np.round(A1,3)

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 3.   , 1.111, 0.845, 0.682, 0.571, 0.492, 0.432],
       [3.   , 1.111, 0.682, 0.492, 0.385, 0.316, 0.268, 0.233],
       [3.   , 0.845, 0.492, 0.347, 0.268, 0.218, 0.184, 0.159],
       [3.   , 0.682, 0.385, 0.268, 0.205, 0.167, 0.14 , 0.121],
       [3.   , 0.571, 0.316, 0.218, 0.167, 0.135, 0.113, 0.098],
       [3.   , 0.492, 0.268, 0.184, 0.14 , 0.113, 0.095, 0.082],
       [3.   , 0.432, 0.233, 0.159, 0.121, 0.098, 0.082, 0.07 ]])

In [ ]:
np.linalg.matrix_rank(A1)

8

In [ ]:
L, U= diy_lu(A1)

print(L, U)

[[nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]] [[nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan]]


<ipython-input-1-8918e3eddeb9>:15: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
<ipython-input-1-8918e3eddeb9>:17: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
<ipython-input-1-8918e3eddeb9>:20: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-1-8918e3eddeb9>:15: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]


The LU decomposition from scipy.linalg.lu already implements pivoting other sophisticated controls 

https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html

$$A = P L U$$

```python

 P ,  L,  U  = scipy.linalg.lu(a, permute_l=False, overwrite_a=False, check_finite=True)

# Returns
# (If permute_l == False)
# P : Permutation matrix
# L : Lower triangular or trapezoidal matrix with unit diagonal. K = min
# U : Upper triangular or trapezoidal matrix
```

In [ ]:
from scipy import linalg
P ,  L,  U = linalg.lu(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",np.round(U,3), "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")


P
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]] 

L
 [[ 1.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.53667954  1.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.73544974  0.70075758  1.          0.          0.
   0.          0.        ]
 [ 1.          0.9025974   0.29108392  0.67870879  1.          0.
   0.          0.        ]
 [ 1.          0.83903421  0.46132533  0.92850281  0.89167088  1.
   0.          0.        ]
 [ 1.          0.9765808   0.07311183  0.19522349  0.39248013 -0.45677246
   1.          0.        ]
 [ 1.          0.94557823  0.16691729  0.42268872  0.75772008 -0.52514217
   0.91395946  1.        ]] 

U
 [[ 3.000e

In [ ]:
P ,  L,  U = linalg.lu(A1)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",np.round(U,3), "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")


P
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]] 

L
 [[ 1.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.         -0.          1.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.73544974  0.14975335  1.          0.          0.
   0.          0.        ]
 [ 1.          0.83903421  0.09858618  0.59429463  1.          0.
   0.          0.        ]
 [ 1.          0.94557823  0.03567057  0.19447289  0.63178599  1.
   0.          0.        ]
 [ 1.          0.9025974   0.06220524  0.35298006  0.92969579  0.9122376
   1.          0.        ]
 [ 1.          0.9765808   0.01562415  0.08279253  0.30466678  0.60602007
  -0.45086164  1.        ]] 

U
 [[ 3.000e+

### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [ ]:
def leading_minors_test(A):
    """
     Function to check all leading Minors of a Matrix not equal to 0
    """
    # ... ENTER YOUR CODE HERE ...

    for i in range(A.shape[0]):
      # Submatrix generation of i dimension
      subMatrixIDim = A[0:i,0:i]
      if np.linalg.det(subMatrixIDim) == 0:
        # Det of submatrix is 0
        return False
    return True

leading_minors_test(A), leading_minors_test(A1)

(True, False)

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

# 2. $LU$ factorization column pivoting and reconstruction
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and then premultply by lower triangular elemetary matrices $\Lambda_j$, $A$  it is transformed in an  upper triangular matrix $U$

$$  
(\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)A  = U
$$
and 
$$  
A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U
$$


So the $LU$ column pivot factorization of $A$ is 
$$
A = LU
$$
ith
$$  
U = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1) A
$$
$$
\begin{array}{ll}L  &=  (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1}\\
&= P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\\
 &= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
\end{array}
$$
because the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$.

Note that 
$$
L= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
$$
is not exactly a lower tiangular matrix but a row permutated lower tiangular matrix.





$$PA = LU$$

In [ ]:
from math import fabs
def diy_lu_column_pivot_reconstruct(A):
    """
     Construct the LU decomposition of the input matrix.
     LU decomposition with pivot: work column by column, accumulate elementary triangular matrices L @ np.transpose(Pj) .
    """
    # ... ENTER YOUR CODE HERE ...

    # Tamaño n
    N = A.shape[0]  
    # Copia en u
    u = A.copy()
    # Dos identidades de nxn
    L = np.eye(N) 
    P = np.eye(N)  

    for j in range(N-1):
        # Inicializar pivote
        pivotValue = 0
        pivotIndex = -1
        # Pivotear en u por fila
        for row in range(j, len(u)):
            # si el valor absoluto es mayor que el pivote actual actualizar pivote
            if fabs(u[row][j]) > pivotValue:     
                pivotValue = fabs(u[row][j])
                pivotIndex = row
        
        if pivotValue != 0:
            # Copias
            u_copy = u.copy()
            L_copy = L.copy()
            # Identidad y copia para permutar
            P_temp = np.eye(N)
            P_copy = P_temp.copy()
            # Reemplazar por pivote en las matrices l, u, p
            L[j, :j], L[pivotIndex, :j] = L_copy[pivotIndex, :j], L_copy[j, :j]
            u[j], u[pivotIndex] = u_copy[pivotIndex], u_copy[j]
            P_temp[j], P_temp[pivotIndex] = P_copy[pivotIndex], P_copy[j]
            # Multiplicar p por la identidad temporal permutada
            P = P@P_temp

        # Identidad 
        lam = np.eye(N)
        # Valor gamma 
        gamma = u[j+1:, j] / u[j, j] 
        # Valor -gamma en identidad
        lam[j+1:, j] = -gamma
        # Multiplicar u por identidad con valor -gamma
        u = lam @ u  
        # Valor gamma en identidad                   
        lam[j+1:, j] = gamma
        # Multiplicar l por identidad con valor -gamma
        L = L @ lam

    return  P, L , u

In [ ]:
A = np.array([[4,9,1], [15,7,0], [19,5,2]])

P, L, U, = diy_lu_column_pivot_reconstruct(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]] 

L
 [[1.         0.         0.        ]
 [0.21052632 1.         0.        ]
 [0.78947368 0.38410596 1.        ]] 

U
 [[19.          5.          2.        ]
 [ 0.          7.94736842  0.57894737]
 [ 0.          0.         -1.8013245 ]] 

A= P@L@U
 [[ 4.  9.  1.]
 [15.  7.  0.]
 [19.  5.  2.]] 

A
 [[ 4  9  1]
 [15  7  0]
 [19  5  2]] 

L@u - A
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 



In [ ]:
from scipy import linalg

A = np.array([[4,9,1], [15,7,0], [19,5,2]])

P,L,U = linalg.lu(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]] 

L
 [[1.         0.         0.        ]
 [0.21052632 1.         0.        ]
 [0.78947368 0.38410596 1.        ]] 

U
 [[19.          5.          2.        ]
 [ 0.          7.94736842  0.57894737]
 [ 0.          0.         -1.8013245 ]] 

A= P@L@U
 [[ 4.  9.  1.]
 [15.  7.  0.]
 [19.  5.  2.]] 

A
 [[ 4  9  1]
 [15  7  0]
 [19  5  2]] 

L@u - A
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 



In [ ]:
N = 8
A = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        A[i, j] = 3. / (0.85*i*j + 1)

P, L, U, = diy_lu_column_pivot_reconstruct(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")




P
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]] 

L
 [[ 1.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.53667954  1.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.73544974  0.70075758  1.          0.          0.
   0.          0.        ]
 [ 1.          0.9025974   0.29108392  0.67870879  1.          0.
   0.          0.        ]
 [ 1.          0.83903421  0.46132533  0.92850281  0.89167088  1.
   0.          0.        ]
 [ 1.          0.9765808   0.07311183  0.19522349  0.39248013 -0.45677246
   1.          0.        ]
 [ 1.          0.94557823  0.16691729  0.42268872  0.75772008 -0.52514217
   0.91395947  1.        ]] 

U
 [[ 3.0000

In [ ]:
from scipy import linalg

P,L,U = linalg.lu(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]] 

L
 [[ 1.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.53667954  1.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.73544974  0.70075758  1.          0.          0.
   0.          0.        ]
 [ 1.          0.9025974   0.29108392  0.67870879  1.          0.
   0.          0.        ]
 [ 1.          0.83903421  0.46132533  0.92850281  0.89167088  1.
   0.          0.        ]
 [ 1.          0.9765808   0.07311183  0.19522349  0.39248013 -0.45677246
   1.          0.        ]
 [ 1.          0.94557823  0.16691729  0.42268872  0.75772008 -0.52514217
   0.91395946  1.        ]] 

U
 [[ 3.0000

In [ ]:
A[1, 1] = 3

P, L, U, = diy_lu_column_pivot_reconstruct(A1)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]] 

L
 [[ 1.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.          1.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.73544974  0.14975335  1.          0.          0.
   0.          0.        ]
 [ 1.          0.83903421  0.09858618  0.59429463  1.          0.
   0.          0.        ]
 [ 1.          0.94557823  0.03567057  0.19447289  0.63178599  1.
   0.          0.        ]
 [ 1.          0.9025974   0.06220524  0.35298006  0.92969579  0.9122376
   1.          0.        ]
 [ 1.          0.9765808   0.01562415  0.08279253  0.30466678  0.60602007
  -0.45086163  1.        ]] 

U
 [[ 3.00000

In [ ]:
from scipy import linalg

P,L,U = linalg.lu(A)

print("P\n",P, "\n")
print("L\n",L, "\n")
print("U\n",U, "\n")
print("A= P@L@U\n", P@L@U, "\n")
print("A\n",A, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - A\n",np.round(P@L@U-A,3), "\n")

P
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]] 

L
 [[ 1.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.          1.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.         -0.          1.          0.          0.          0.
   0.          0.        ]
 [ 1.          0.73544974  0.14975335  1.          0.          0.
   0.          0.        ]
 [ 1.          0.83903421  0.09858618  0.59429463  1.          0.
   0.          0.        ]
 [ 1.          0.94557823  0.03567057  0.19447289  0.63178599  1.
   0.          0.        ]
 [ 1.          0.9025974   0.06220524  0.35298006  0.92969579  0.9122376
   1.          0.        ]
 [ 1.          0.9765808   0.01562415  0.08279253  0.30466678  0.60602007
  -0.45086164  1.        ]] 

U
 [[ 3.00000